In [1]:
# Importando as bibliotecas
import ipywidgets as widgets  # Para criar os botões interativos para o Jogo da Velha
import pyspark as spark  # Para manipulação e processamento de dados com PySpark
import matplotlib.pyplot as plt  # Para visualizações gráficas (não utilizado diretamente aqui)
import numpy as np  # Para operações numéricas (não utilizado diretamente aqui)
import pandas as pd  # Para manipulação de dados (não utilizado diretamente aqui)
import seaborn as sns  # Para visualizações estatísticas (não utilizado diretamente aqui)
from IPython.display import display  # Para exibir widgets interativos no Jupyter Notebook
from pyspark.sql import SparkSession  # Para criar uma SparkSession para usar o PySpark
from google.colab import files

In [2]:
# Criação da SparkSession
spark = SparkSession.builder \
    .appName("jogodavelha") \
    .getOrCreate()

In [7]:
uploaded = files.upload()  # Você faz o upload do arquivo
codigo = list(uploaded.keys())[0]  # Pega o nome do arquivo, por exemplo, 'jogo.txt'
with open(codigo, 'r') as file:
    criar_tabuleiro = file.read()  # Lê o conteúdo de 'jogo.txt' e armazena em criar_tabuleiro

Saving codigo.txt to codigo (1).txt


In [24]:
# Função para criar o tabuleiro do Jogo da Velha
def criar_tabuleiro():
    """
    Cria e exibe o tabuleiro interativo do Jogo da Velha utilizando o ipywidgets.
    O tabuleiro é composto por 9 botões, um para cada célula do jogo.
    Cada vez que um botão é pressionado, o jogo alterna entre os jogadores 'X' e 'O'.
    Além disso, verifica se algum jogador venceu ou se houve empate.
    """
    # Criando os botões do tabuleiro
    botoes = [widgets.Button(description=' ') for _ in range(9)]
    for i in range(9):
        botoes[i].layout.width = '100px'
        botoes[i].layout.height = '100px'
        botoes[i].layout.border = '1px solid black'  # Adicionando borda aos botões

    # Função para fazer a jogada
    def jogar(botao, index):
        """
        Função que alterna entre os jogadores 'X' e 'O' e realiza a jogada no botão pressionado.
        Se o botão estiver vazio, ele será preenchido com 'X' ou 'O', dependendo do jogador atual.
        Em seguida, chama a função verificar_vencedor().
        """
        if botao.description == ' ':
            if jogador_atual[0] == 'X':
                botao.description = 'X'
                botao.style.button_color = 'Purple'
                jogador_atual[0] = 'O'
            else:
                botao.description = 'O'
                botao.style.button_color = 'Green'
                jogador_atual[0] = 'X'
            verificar_vencedor(botoes)

    # Função para verificar se algum jogador venceu
    def verificar_vencedor(botoes):
        """
        Verifica se algum jogador venceu o jogo verificando todas as combinações de vitória.
        Se houver um vencedor, imprime uma mensagem indicando o vencedor.
        Caso todos os botões sejam preenchidos e não haja vencedor, imprime "Empate!".
        """
        combinacoes_vitoria = [(0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6)]
        for c in combinacoes_vitoria:
            if botoes[c[0]].description == botoes[c[1]].description == botoes[c[2]].description != ' ':
                vencedor = botoes[c[0]].description
                salvar_resultado(vencedor)
                for botao in botoes:
                    botao.disabled = True
                return
        if all(botao.description != ' ' for botao in botoes):
            salvar_resultado("Empate!")
            return

    # Definindo as ações de clique nos botões
    for i in range(9):
        botoes[i].on_click(lambda x, index=i: jogar(x, index))

    # Exibindo o tabuleiro no formato de grade
    grid = widgets.GridBox(
        children=botoes,
        layout=widgets.Layout(
            grid_template_columns="repeat(3, 100px)",
            grid_template_rows="repeat(3, 100px)",
            grid_gap='2px',  # Espaço entre os botões
            border='2px solid black'  # Borda ao redor do tabuleiro
        )
    )

    display(grid)

def salvar_resultado(resultado):
    """
    Função para salvar o resultado do jogo em um arquivo de texto.
    """
    # Criando uma pasta chamada 'n_resultado_jogo' se não existir
    !mkdir -p /content/resultado_jogo

    # Caminho do arquivo onde o resultado será salvo
    caminho_arquivo = "/content/resultado_jogo/resultado_jogo.txt"

    # Salvar o resultado em um arquivo de texto
    with open(caminho_arquivo, 'w') as file:
        file.write(f"O jogador vencedor foi: {resultado}")

# Jogador atual
jogador_atual = ['X']

# Criando e exibindo o tabuleiro
criar_tabuleiro()

GridBox(children=(Button(description=' ', layout=Layout(border='1px solid black', height='100px', width='100px…